In [ ]:
import numpy as np
import pandas as pd
import pickle
import pandas.io.sql as pdsql
import psycopg2

from mpl_toolkits.basemap import Basemap

import matplotlib as mpl
import pylab as pl
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import LineCollection, PatchCollection

import imageio

#%matplotlib inline
axis_font = {'size':'30'}
mpl.rcParams['xtick.labelsize'] = 30
mpl.rcParams['ytick.labelsize'] = 30

In [ ]:
df = pickle.load(open("./intermediate_files/dataframes_visualize_nta/data_raw_nta.p", "rb"))

conn = psycopg2.connect("dbname='nyc_taxi' user='postgres' host='localhost' password='organon'")
nta = pdsql.read_sql("SELECT gid, shape_area, ntacode, ntaname FROM nta", conn, coerce_float=True, params=None)
day_info = pdsql.read_sql("SELECT doy, holiday FROM day_info", conn, coerce_float=True, params=None)
conn.close()

In [ ]:
# The index of the NTAS are ntacode and gid
ntacode2gid = {nta.ntacode[i]: nta.gid[i] for i in range(len(nta.gid))}

In [ ]:
# The hourly pickup per square mile for each NTA averaged over 365 days of 2014
sqft2sqmi=3.58701e-8
density_hourly = np.empty((195, 24), dtype='float64')
for gid in range(1, 196):
    density_hourly[gid-1, :] = np.array([np.mean(df['count'][(df.pickup_hour==h) & (df.pickup_gid==gid)]) for h in range(24)])\
                            / nta.shape_area[nta.gid==gid].values[0] / sqft2sqmi

In [ ]:
# Plot the 24-hour density map
density_hourly_max = np.max(density_hourly)
west, south, east, north = -74.15, 40.50, -73.65, 40.95

for hour in range(24):
    m = Basemap(llcrnrlon=west, llcrnrlat=south, urcrnrlon=east, urcrnrlat=north,
             resolution='i', projection='merc', lat_0=(south+north)/2, lon_0=(west+east)/2)
    fig = plt.figure(figsize=(30,30))
    ax = fig.add_subplot(111)
    ax.set_title('h={0}'.format(hour), fontsize=35)

    m.readshapefile('./datasets/nta_2010/geo_export_d2bec86b-a3e8-4047-b785-24286f7718ab', 'nyc', linewidth=1, zorder=1)

    patches = []
    cm = mpl.cm.jet
    for info, shape in zip(m.nyc_info, m.nyc):
        patches.append(Polygon(np.array(shape), True,
                               facecolor=cm(density_hourly[ntacode2gid[info['ntacode']]-1, hour] / density_hourly_max)))
    
    p = PatchCollection(patches, match_original=True, alpha=0.6, linewidths=1., zorder=2)

    ax.add_collection(p)

    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.12, 0.02, 0.78])
    a = np.array([[0.0,1.0]])

    pl.figure(figsize=(9, 1.5))
    img = pl.imshow(a, cmap=mpl.cm.jet, alpha=0.6)
    pl.gca().set_visible(False)
    cbar = fig.colorbar(img, ticks=np.arange(0.0, 1.01, 0.2), orientation='vertical', cax =cbar_ax)
    cbar.ax.set_yticklabels((np.arange(0.0, 1.01, 0.2) * density_hourly_max).astype('int')) 
    cbar.ax.set_ylabel('pickup/hr/mi$^2$', **axis_font)

    fig.savefig('density_{0}.jpg'.format(hour), dpi=50, bbox_inches='tight')
    plt.close()

In [ ]:
# Generate gif file
images = []
for hour in range(24):
    images.append(imageio.imread('density_{0}.jpg'.format(hour)))
imageio.mimsave('./figures/density_movie_nta.gif', images, fps=0.5)

In [ ]:
# Time series analysis for the daily number of pickups in NYC
count_total_hourly = np.sum(df['count'].values.reshape((195, 365 * 24)), axis=0)
fft_count_total_hourly = np.fft.fft(count_total_hourly)

In [ ]:
fig, ax = plt.subplots(figsize=(30, 6))
ax.plot(np.arange(365 * 24) / 24, count_total_hourly, color='k', lw=2, linestyle='-')
holidays = day_info.doy[day_info.holiday].values
for day in holidays:
    ax.axvline(day+0.5, color='blue', linestyle='--', linewidth=3)
ax.axvline(holidays[0]+0.5, color='blue', linestyle='--', linewidth=3, label='Holidays')
ax.set_xlabel('Day of the year', **axis_font)

ax.set_ylabel('Total Pickup', **axis_font)
ax.legend(loc=0, fontsize=30)
ax.grid(True)

fig.savefig('./figures/total_pickup_time.jpg', dpi=50, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(30, 6))
ax.plot(np.arange(365 * 24 / 2) / 365, np.abs(fft_count_total_hourly[0:int(len(fft_count_total_hourly)/2)]), color='k', lw=2, linestyle='-')
ax.set_xlabel('Frequency (day$^{-1}$)', **axis_font)
ax.axvline(1, color='blue', linestyle='--', linewidth=3, label='Daily')
ax.axvline(1/7, color='red', linestyle='--', linewidth=3, label='Weekly')
ax.set_ylabel('FFT(Total Pickup)', **axis_font)
ax.set_ylim([0, 5e7])
ax.legend(loc=4, fontsize=30)
ax.grid(True)
fig.savefig('./figures/total_pickup_freq.jpg', dpi=50, bbox_inches='tight')